# 下在训练数据

In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!wget http://www.robots.ox.ac.uk/~vgg/data/text/mjsynth.tar.gz 

--2019-06-04 14:21:06--  http://www.robots.ox.ac.uk/~vgg/data/text/mjsynth.tar.gz
Resolving www.robots.ox.ac.uk (www.robots.ox.ac.uk)... 129.67.94.2
Connecting to www.robots.ox.ac.uk (www.robots.ox.ac.uk)|129.67.94.2|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10678411583 (9.9G) [application/x-gzip]
Saving to: ‘mjsynth.tar.gz’

mjsynth.tar.gz      100%[===================>]   9.94G  21.1MB/s    in 8m 12s  

2019-06-04 14:29:19 (20.7 MB/s) - ‘mjsynth.tar.gz’ saved [10678411583/10678411583]



In [0]:
!wget http://www.ee.surrey.ac.uk/CVSSP/demos/chars74k/EnglishImg.tgz

--2019-06-04 14:29:26--  http://www.ee.surrey.ac.uk/CVSSP/demos/chars74k/EnglishImg.tgz
Resolving www.ee.surrey.ac.uk (www.ee.surrey.ac.uk)... 131.227.80.48
Connecting to www.ee.surrey.ac.uk (www.ee.surrey.ac.uk)|131.227.80.48|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 133975105 (128M) [application/x-gzip]
Saving to: ‘EnglishImg.tgz’

EnglishImg.tgz      100%[===================>] 127.77M  19.6MB/s    in 7.3s    

2019-06-04 14:29:34 (17.4 MB/s) - ‘EnglishImg.tgz’ saved [133975105/133975105]



In [0]:
!tar -xzf mjsynth.tar.gz

In [0]:
!tar -xzf EnglishImg.tgz

In [0]:
import os
import string
target_data_file = '/content/mnt/ramdisk/max/90kDICT32px/annotation_singleletter.txt'
badimgpath = '/content/English/Img/BadImag/Bmp/'
goodimgpath = '/content/English/Img/GoodImg/Bmp/'
numberandletter = string.digits + string.ascii_uppercase + string.ascii_lowercase

i=0
with open(target_data_file, 'w') as t:
    for s in numberandletter:
        i=i+1
        m = "%03d" % i
        for p in os.listdir(badimgpath+'Sample'+m):
            t.write(badimgpath+'Sample'+m+"/"+p+" " + s +'\n')
            
        for p in os.listdir(goodimgpath+'Sample'+m):
            t.write(goodimgpath+'Sample'+m+'/'+p+" " + s +'\n')   

In [0]:
cd /content/mnt/ramdisk/max/90kDICT32px/

/content/mnt/ramdisk/max/90kDICT32px


In [0]:
import os
from PIL import Image
synth90k_data_file = '/content/mnt/ramdisk/max/90kDICT32px/annotation_train.txt'
target_data_file =  '/content/mnt/ramdisk/max/90kDICT32px/annotation_synth90k.txt'

i=0
with open(synth90k_data_file, 'r') as f, open(target_data_file, 'w') as t:
    for line in f:
        if i%10==0:
            parts = line.split(" ")
            filename = parts[0]
            try:
              Image.open(filename)
            except OSError:
              continue
            parts = filename.split("_")
            t.write(filename+" "+parts[1]+'\n')
        i=i+1

In [0]:
!cat /content/mnt/ramdisk/max/90kDICT32px/annotation_singleletter.txt /content/mnt/ramdisk/max/90kDICT32px/annotation_synth90k.txt > /content/mnt/ramdisk/max/90kDICT32px/annotation_train_90k_sl.txt

In [0]:
!shuf /content/mnt/ramdisk/max/90kDICT32px/annotation_train_90k_sl.txt > /content/mnt/ramdisk/max/90kDICT32px/annotation_train_90k_sl_shuffled.txt

# 安装acor, 准备tfrecords文件

In [0]:
pip install aocr

     |████████████████████████████████| 184kB 13.5MB/s 
  Stored in directory: /root/.cache/pip/wheels/cd/45/bc/530a415bf7cabe48545bf209c2e2c09c1849e20ea37d413ded
  Stored in directory: /root/.cache/pip/wheels/d5/aa/e1/dbba9e7b6d397d645d0f12db1c66dbae9c5442b39b001db18e
Successfully built aocr distance


In [0]:
cd /content/mnt/ramdisk/max/90kDICT32px/

/content/mnt/ramdisk/max/90kDICT32px


In [0]:
!nohup aocr dataset --no-force-uppercase /content/mnt/ramdisk/max/90kDICT32px/annotation_train_90k_sl_shuffled.txt /content/annotation_train_90k_sl_shuffled.tfrecords &

nohup: appending output to 'nohup.out'


# 进行训练

In [0]:
!nohup aocr train --full-ascii --color --max-width 500 --max-height 200 --max-prediction 30 --target-embedding-size 30 --output-dir "/content/gdrive/My Drive/aocr-result" --model-dir "/content/gdrive/My Drive/aocr-result" --num-epoch 100 --batch-size 2000 /content/annotation_train_90k_sl_shuffled.tfrecords &

nohup: appending output to 'nohup.out'


# 辅助命令

In [0]:
!ps -ef|grep aocr

root        7862       1 21 04:04 ?        00:01:20 /usr/bin/python3 /usr/local/bin/aocr dataset --no-force-uppercase /content/mnt/ramdisk/max/90kDICT32px/annotation_train_90k_sl_shuffled.txt /content/annotation_train_90k_sl_shuffled.tfrecords
root        8077    7664  0 04:11 ?        00:00:00 /bin/bash -c ps -ef|grep aocr
root        8079    8077  0 04:11 ?        00:00:00 grep aocr


In [0]:
!kill -9 7885

In [1]:
!nvidia-smi

Wed Jan  1 08:45:40 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.44       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
!tail /content/mnt/ramdisk/max/90kDICT32px/nohup.out -f

2019-06-05 07:26:51,109 root  INFO     Step 3981: 3.959s, loss: 0.065046, perplexity: 1.067208.
2019-06-05 07:26:55,246 root  INFO     Step 3982: 3.899s, loss: 0.065157, perplexity: 1.067326.
2019-06-05 07:26:59,337 root  INFO     Step 3983: 3.844s, loss: 0.075068, perplexity: 1.077957.
2019-06-05 07:27:03,399 root  INFO     Step 3984: 3.833s, loss: 0.054538, perplexity: 1.056053.
2019-06-05 07:27:07,640 root  INFO     Step 3985: 3.982s, loss: 0.058144, perplexity: 1.059867.
2019-06-05 07:27:11,701 root  INFO     Step 3986: 3.826s, loss: 0.066563, perplexity: 1.068828.
2019-06-05 07:27:15,858 root  INFO     Step 3987: 3.906s, loss: 0.067209, perplexity: 1.069519.
2019-06-05 07:27:19,994 root  INFO     Step 3988: 3.892s, loss: 0.068977, perplexity: 1.071412.
2019-06-05 07:27:24,118 root  INFO     Step 3989: 3.881s, loss: 0.064486, perplexity: 1.066611.
2019-06-05 07:27:28,479 root  INFO     Step 3990: 4.078s, loss: 0.063016, perplexity: 1.065044.
2019-06-05 07:27:32,571 root  INFO     S

In [0]:
!tail /content/nohup.out

In [0]:
!tail /content/mnt/ramdisk/max/90kDICT32px/annotation_train_90k_sl_shuffled.txt -f

./2187/6/107_INBREEDING_38649.jpg INBREEDING
./442/4/125_Corpora_17114.jpg Corpora
./1998/6/158_Firm_29113.jpg Firm
./1724/2/150_GRUNION_34023.jpg GRUNION
./2108/4/84_equipment_26305.jpg equipment
./240/6/12_Megos_47594.jpg Megos
./2/2/170_Beget_6695.jpg Beget
./2179/5/280_sadist_67322.jpg sadist
./477/1/392_buildup_10065.jpg buildup
./1373/2/265_VENTURES_84290.jpg VENTURES
^C


In [0]:
!ps -ef|grep aocr

root        5930       1 20 07:46 ?        00:01:32 /usr/bin/python3 /usr/local/bin/aocr dataset --no-force-uppercase /content/mnt/ramdisk/max/90kDICT32px/annotation_train_90k_sl_shuffled.txt /content/annotation_train_90k_sl_shuffled.tfrecords
root        6133    3372  0 07:54 ?        00:00:00 /bin/bash -c ps -ef|grep aocr
root        6135    6133  0 07:54 ?        00:00:00 grep aocr


In [0]:
!ls -alh

total 11G
drwxr-xr-x 1 root root 4.0K Jun  3 07:15 .
drwxr-xr-x 1 root root 4.0K Jun  3 06:51 ..
-rw-r--r-- 1 root root 100M Jun  3 07:47 annotation_train_90k_sl_shuffled.tfrecords
-rw-r--r-- 1 root root  13M Jun  3 07:44 aocr.log
drwxr-xr-x 2 root root 4.0K Jun  3 07:00 checkpoints
drwxr-xr-x 1 root root 4.0K May 30 16:09 .config
drwxr-xr-x 3 root root 4.0K Jun  3 04:36 English
-rw-r--r-- 1 root root 128M Aug 27  2009 EnglishImg.tgz
drwx------ 3 root root 4.0K Jun  3 03:03 gdrive
drwxr-xr-x 2 root root 4.0K Jun  3 07:14 .ipynb_checkpoints
-rw-r--r-- 1 root root  10G Dec  7  2014 mjsynth.tar.gz
drwxr-xr-x 3 root root 4.0K Jun  3 03:32 mnt
-rw------- 1 root root 136K Jun  3 07:44 nohup.out
drwxr-xr-x 1 root root 4.0K May 24 16:08 sample_data


In [0]:
!wc -l /content/mnt/ramdisk/max/90kDICT32px/annotation_singleletter.txt /content/mnt/ramdisk/max/90kDICT32px/annotation_synth90k.txt

   12503 /content/mnt/ramdisk/max/90kDICT32px/annotation_singleletter.txt
  722462 /content/mnt/ramdisk/max/90kDICT32px/annotation_synth90k.txt
  734965 total


In [0]:
!ls /content -alh

total 12G
drwxr-xr-x 1 root root 4.0K Jun  4 03:10 .
drwxr-xr-x 1 root root 4.0K Jun  4 01:15 ..
-rw-r--r-- 1 root root 1.3G Jun  4 03:25 annotation_train_90k_sl_shuffled.tfrecords
drwxr-xr-x 1 root root 4.0K May 31 16:17 .config
drwxr-xr-x 3 root root 4.0K Jun  4 01:58 English
-rw-r--r-- 1 root root 128M Aug 27  2009 EnglishImg.tgz
drwx------ 3 root root 4.0K Jun  4 02:50 gdrive
-rw-r--r-- 1 root root  10G Dec  7  2014 mjsynth.tar.gz
drwxr-xr-x 3 root root 4.0K Jun  4 01:22 mnt
drwxr-xr-x 1 root root 4.0K May 31 16:17 sample_data


In [0]:
cd /content/gdrive/My Drive/aocr-result/

/content/gdrive/My Drive/aocr-result


In [0]:
from google.colab import files

files.download('model.ckpt-20100.data-00000-of-00001')

MessageError: ignored